In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import os, sys
sys.path.insert(0, os.path.join(os.getenv('HOME'), 'StarNet'))
import numpy as np
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
from astropy.io import fits as pyfits
import umap
from sklearn.datasets import load_iris
"""
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
"""
from starnet.utils.data_utils.augment import convolve_spectrum
from starnet.utils.data_utils.restructure_spectrum import rebin, continuum_normalize, ensure_constant_sampling


# In[2]:

/home/nrpu88/jupyter_py3/lib/python3.6/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/home/nrpu88/jupyter_py3/lib/python3.6/site-packages/pysynphot/locations.py:329: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))
/home/nrpu88/jupyter_py3/lib/python3.6/site-packages/pysynphot/refs.py:118: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for ftp://ftp.stsci.edu/cdbs/mtab/*_tmg.fits
  'functionality will be SEVERELY crippled. ' + str(e))
/home/nrpu88/jupyter_py3/lib/python3.6/site-packages/pysynphot/refs.py:125: UserWarning: No thermal tables found, no thermal calculations can be performed. No files found for ftp://ftp.stsci.edu/cdbs/mtab/*_tmt.fits
  'no thermal calculations can be performed. ' + str(e))


In [2]:
# Define parameters needed for continuum fitting
LINE_REGIONS = [[4210, 4240], [4250, 4410], [4333, 4388], [4845, 4886], [5160, 5200], [5874, 5916], [6530, 6590]]
SEGMENTS_STEP = 10.  # divide the spectrum into segments of 10 Angstroms


# In[3]:


home = os.getenv('HOME')
scratch = os.getenv('SCRATCH')
starnet_data_folder = os.path.join(home, 'StarNet/starnet/data/')
intrigoss_grid_path = os.path.join(home, 'projects/rrg-kyi/group_writable/spectra/grids/intrigoss/train/') 
phoenix_grid_path = os.path.join(home, 'projects/rrg-kyi/group_writable/spectra/grids/phoenix.astro.physik.uni-goettingen.de/v2.0/HiResFITS/PHOENIX-ACES-AGSS-COND-2011/train/') 
phoenix_wave_path = home+'/'+'/projects/rrg-kyi/group_writable/spectra/grids/phoenix.astro.physik.uni-goettingen.de/v2.0/HiResFITS/PHOENIX-ACES-AGSS-COND-2011/'
ambre_grid_path = os.path.join(home, 'projects/rrg-kyi/group_writable/spectra/grids/AMBRE/train/')
obs_wave_filepath = os.path.join(home, 'projects/rrg-kyi/group_writable/spectra/UVES_4835-5395.npy')
wave_grid_obs = np.load(obs_wave_filepath)

#print(wave_grid_obs)
#print(len(wave_grid_obs))
#wave_grid_obs = np.linspace(4835,5395,num=10000)


# In[4]:

In [3]:
def get_phoenix_spectrum(spectrum_path, wave_grid_path):
    
    """
    Given the path of a Phoenix spectrum .fits file, this function retrieves the flux and wavelength data
    
    INPUT: path: The path to the Phoenix spectrum file, e.g. '/path/to/lte04000-1.00-1.0.Alpha=+0.50.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits'
    
    RETURNS: wav: wavelength data
             flux: flux data
    """
    
    hdulist_spectrum = pyfits.open(spectrum_path)
    hdulist_wavegrid = pyfits.open(wave_grid_path)
    
    flux = hdulist_spectrum[0].data
    wav = hdulist_wavegrid[0].data
    
    # For Phoenix, need to convert from vacuum to air wavelengths.
    # The IAU standard for conversion from air to vacuum wavelengths is given
    # in Morton (1991, ApJS, 77, 119). For vacuum wavelengths (VAC) in
    # Angstroms, convert to air wavelength (AIR) via:
    #  AIR = VAC / (1.0 + 2.735182E-4 + 131.4182 / VAC^2 + 2.76249E8 / VAC^4)
    wav = wav / (
            1.0 + 2.735182E-4 + 131.4182 / wav ** 2 + 2.76249E8 / wav ** 4)
    
    return wav, flux


# In[5]:


def get_phoenix_filename(teff, logg, feh, afe):
    """
    This function returns the name of the Phoenix spectrum file with the requested stellar parameters.
    
    INPUT: teff: Effective temperature (K)
           logg: Surface gravity
           feh: Metallicity (dex)
           afe: Alpha elements abundance [alpha/Fe]

    RETURNS: filename: Name of the .fits file containing an INTRIGOSS spectrum
    EXAMPLE: With input of teff=4000, logg=1.0, feh=-1.0, afe=0.5, it returns the string
             'lte04000-1.00-1.0.Alpha=+0.50.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits'
    """
    filename = ''    
    str_teff = 'lte{:05d}'.format(teff)
    str_logg = '-{:03.2f}'.format(logg)
    str_feh = '{:02.1f}'.format(feh)
    if feh>0: 
        str_feh = '+' + str_feh
        
    if afe == 0:
        str_afe = ''
    elif afe < 0:
        str_afe = '.Alpha={:03.2f}'.format(afe)
    elif afe > 0:
        str_afe = '.Alpha=+{:03.2f}'.format(afe)

    filename = '{}{}{}{}.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits'.format(str_teff, str_logg, str_feh, str_afe)
    
    if teff <= 5000 or logg <=3 or not(-1 < feh < 0):
        filename = ''

    return filename
           
    
    


# In[6]:


def get_intrigoss_spectrum(path):
    
    """
    Given the path of an INTRIGOSS spectrum .fits file, this function retrieves the flux and wavelength data
    
    INPUT: path: The path to the INTRIGOSS spectrum file, e.g. '/path/to/alpp050_am100_t4000g10_v1_f.fits'
    
    RETURNS: wav: wavelength data
             flux: flux data
    """
    
    hdulist = pyfits.open(path)
    flux_data = hdulist[1].data
    
    wav = flux_data['wavelength']
    flux = flux_data['surface_flux']
    
    return wav, flux


# In[7]:


def get_ambre_spectrum(path):
    
    """
    Given the path of an AMBRE spectrum .fits file, this function retrieves the flux and wavelength data
    
    INPUT: path: The path to the AMBRE spectrum file, e.g. '/path/to/alpp050_am100_t4000g10_v1_f.AMBRE'
    
    RETURNS: wav: wavelength data
             flux: flux data
    """
    flux = np.genfromtxt(path,usecols=-1)
    wav = np.genfromtxt(path,usecols= 0)
    
    return wav, flux


# In[8]:


def get_ambre_filename(teff, logg, feh, afe):
    
    """
    This function returns the name of the AMBRE spectrum file with the requested stellar parameters.
    
    INPUT: teff: Effective temperature (K)
           logg: Surface gravity
           feh: Metallicity (dex)
           afe: Alpha elements abundance [alpha/Fe]

    RETURNS: filename: Name of the .fits file containing an AMBRE spectrum
    EXAMPLE: With input of teff=4000, logg=1.0, feh=-1.0, afe=0.5, it returns the string
             'alpp050_am100_t4000g10_v1_f.AMBRE'
    """
    filename_fin = ''
    # Construct the full AMBRE .fits file name from stellar parameters
    for root, dirs, files in os.walk(ambre_grid_path):
        for filename in files:
            if teff == float(filename[1:5]):
                if filename[7] == '-':
                    if logg == -1*float(filename[8:11]):
                        if filename[22] == '-':
                            if feh == -1*float(filename[23:27]):
                                if filename[29] == '-':
                                    if afe == -1*float(filename[30:34]):
                                        filename_fin = filename
                                elif afe == float(filename[30:34]):
                                    filename_fin = filename
                                    
                        elif feh == float(filename[23:27]):
                            if filename[29] == '-':
                                if afe == -1*float(filename[30:34]):
                                    filename_fin = filename
                            elif afe == float(filename[30:34]):
                                filename_fin = filename
    
                        
                elif logg == float(filename[8:11]):
                    if filename[22] == '-':
                        if feh == -1*float(filename[23:27]):
                            if filename[29] == '-':
                                if afe == -1*float(filename[30:34]):
                                    filename_fin = filename
                            elif afe == float(filename[30:34]):
                                    filename_fin = filename
                                    
                    elif feh == float(filename[23:27]):
                        if afe == -1*float(filename[30:34]):
                            filename_fin = filename
                        elif afe == float(filename[30:34]):
                            filename_fin = filename
    
    if teff <= 5000 or logg <=3 or not(-1 < feh < 0):
        filename_fin = ''   
        
    return filename_fin


# In[9]:




def find_closest_ambre_match(teff, logg, feh, afe):
    
    """
    Given a set of stellar parameters, this function will return the stellar parameters from the AMBRE
    grid which are closest to the supplied parameters.
    
    INPUT: teff: Effective temperature (K)
           logg: Surface gravity
           feh: Metallicity (dex)
           afe: Alpha elements abundance [alpha/Fe]

    RETURNS: Closest matching stellar parameters from the AMBRE grid
    EXAMPLE: With input of teff=3900, logg=1.1, feh=-1.1, afe=0.6, it returns 4 values:
             4000, 1.0, -1.0, 0.5
    """
    
        # AMBRE grid spacing
    if teff <= 3900:
        teff_grid = np.arange(2500, 4000, 200)
    elif teff >= 4000:
        teff_grid = np.arange(4000, 8500, 250)
    logg_grid = np.arange(-0.5, 6, 0.5)
    if feh >= -1.0:
        feh_grid = np.arange(-1.5, 1.5, 0.25)
    elif feh >= -3.0 and feh < -1.0:
        feh_grid = np.arange(-3.5,-1.0, 0.5)
    elif feh < -3.0:
        feh_grid = np.arange(-5.0,-3.0, 1.0)
    afe_grid = np.arange(-0.40, 1.00, 0.20) 
    
    
    # Find closest parameter values
    match_teff = teff_grid[np.argmin(np.abs(teff - teff_grid))]
    match_logg = logg_grid[np.argmin(np.abs(logg - logg_grid))]
    match_feh = feh_grid[np.argmin(np.abs(feh - feh_grid))]
    match_afe = afe_grid[np.argmin(np.abs(afe - afe_grid))]
    
    match_afe = round(match_afe,2)
    match_logg = round(match_logg,2)
    match_feh = round(match_feh,2)
    
    return match_teff, match_logg, match_feh, match_afe


# In[11]:


def find_closest_phoenix_match(teff, logg, feh, afe):
    
    """
    Given a set of stellar parameters, this function will return the stellar parameters from the Phoenix
    grid which are closest to the supplied parameters.
    
    INPUT: teff: Effective temperature (K)
           logg: Surface gravity
           feh: Metallicity (dex)
           afe: Alpha elements abundance [alpha/Fe]

    RETURNS: Closest matching stellar parameters from the Phoenix grid
    EXAMPLE: With input of teff=3900, logg=1.1, feh=-1.1, afe=0.6, it returns 4 values:
             3900, 1.0, -1.0, 0.5
    """

    # Phoenix spectra grid spacing
    teff_grid = np.arange(2300,7000,100)
    teff_grid = np.concatenate((teff_grid, np.arange(7000,12001,200)))
    logg_grid = np.arange(0, 6.1, 0.5)
    feh_grid = np.arange(-4., -2.0, 1)
    feh_grid = np.concatenate((feh_grid, np.arange(-2.0, 1.01, 0.5)))
    afe_grid = np.arange(-0.2, 1.21, 0.2)

    # Find closest parameter values
    match_teff = teff_grid[np.argmin(np.abs(teff - teff_grid))]
    match_logg = logg_grid[np.argmin(np.abs(logg - logg_grid))]
    match_feh = feh_grid[np.argmin(np.abs(feh - feh_grid))]
    match_afe = afe_grid[np.argmin(np.abs(afe - afe_grid))]
    
    match_afe = round(match_afe,2)
    
    return match_teff, match_logg, match_feh, match_afe


# In[12]:


def intrigoss_one_file(name):


    wav_intrigoss,flux_intrigoss = get_intrigoss_spectrum(intrigoss_grid_path+name)

    return wav_intrigoss,flux_intrigoss


# In[13]:


def phoenix_one_file(name):


    wavegrid_path = os.path.join(phoenix_wave_path, 'WAVE_PHOENIX-ACES-AGSS-COND-2011.fits')
    wav_phoenix, flux_phoenix = get_phoenix_spectrum(phoenix_grid_path+name, wavegrid_path)


    return wav_phoenix,flux_phoenix


# In[14]:


def ambre_one_file(name):

    wav_ambre, flux_ambre = get_ambre_spectrum(ambre_grid_path+name)
    

    return wav_ambre,flux_ambre





def pre_processor_intrigoss(wav_intrigoss,flux_intrigoss):


# Trim the wavelength and flux arrays according to observed wave grid
    extension = 10  # Angstroms
    wave_min_request = wave_grid_obs[0] - extension
    wave_max_request = wave_grid_obs[-1] + extension
    wave_indices_intrigoss = (wav_intrigoss > wave_min_request) & (wav_intrigoss < wave_max_request)
    #wave_indices_phoenix = (wav_phoenix > wave_min_request) & (wav_phoenix < wave_max_request)
    #wave_indices_ambre = (wav_ambre > wave_min_request) & (wav_ambre < wave_max_request)
    wav_intrigoss = wav_intrigoss[wave_indices_intrigoss]
    #wav_phoenix = wav_phoenix[wave_indices_phoenix]
    #wav_ambre = wav_ambre[wave_indices_ambre]
    flux_intrigoss = flux_intrigoss[wave_indices_intrigoss]
    #flux_phoenix = flux_phoenix[wave_indices_phoenix]
    #flux_ambre = flux_ambre[wave_indices_ambre]

# Degrade resolution
    err_intrigoss = np.zeros(len(flux_intrigoss))
    #err_phoenix = np.zeros(len(flux_phoenix))
    #err_ambre = np.zeros(len(flux_ambre))
    _, flux_intrigoss, _ = convolve_spectrum(wav_intrigoss, flux_intrigoss, err_intrigoss, to_resolution=47000)
    #_, flux_phoenix, _ = convolve_spectrum(wav_phoenix, flux_phoenix, err_phoenix, to_resolution=47000)
    #_, flux_ambre, _ = convolve_spectrum(wav_ambre, flux_ambre, err_ambre, to_resolution=47000)

# Rebin to UVES wave grid
    flux_intrigoss = rebin(wave_grid_obs, wav_intrigoss, flux_intrigoss)
    #flux_phoenix = rebin(wave_grid_obs, wav_phoenix, flux_phoenix)
    #flux_ambre = rebin(wave_grid_obs, wav_ambre, flux_ambre)

# Continuum normalize the spectra
    flux_intrigoss, _ = continuum_normalize(flux_intrigoss, LINE_REGIONS, wave_grid_obs, SEGMENTS_STEP)
    #flux_phoenix, _ = continuum_normalize(flux_phoenix, LINE_REGIONS, wave_grid_obs, SEGMENTS_STEP)
    #flux_ambre, _ = continuum_normalize(flux_ambre, LINE_REGIONS, wave_grid_obs, SEGMENTS_STEP)

    #print('DONE')
    
    return flux_intrigoss

# Mask telluric lines
#flux_intrigoss = mask_tellurics('telluric_lines.txt', flux_intrigoss, wave_grid_obs
#flux_phoenix = mask_tellurics('telluric_lines.txt', flux_phoenix, wave_grid_obs)
#flux_ambre = mask_tellurics('telluric_lines.txt', flux_ambre, wave_grid_obs)

In [4]:
def getIntriParams(file_path):
    with pyfits.open(file_path) as hdulist:
          #flux = hdulist[1].data['surface_flux']
        param_data = hdulist[0].header
        teff = param_data['TEFF']
        logg = param_data['LOG_G']
        m_h = param_data['FEH']
        a_m = param_data['ALPHA']
        
        return teff,logg,m_h,a_m
          #vt = param_data['VT']
          #params = [teff, logg, m_h, a_m, vt]


# In[19]:

'''
import multiprocessing
from contextlib import contextmanager

def pre_processor_intrigoss_parallel(wav, fluxes):

    @contextmanager
    def poolcontext(*args, **kwargs):
            pool = multiprocessing.Pool(*args, **kwargs)
            yield pool
            pool.terminate()

    num_spectra = np.shape(fluxes)[0]
    num_cpu = multiprocessing.cpu_count()
    pool_size = num_cpu if num_spectra >= num_cpu else num_spectra
    print('[INFO] Pool size: {}'.format(pool_size))
    
    pool_arg_list = [(wav, fluxes[i])
                         for i in range(num_spectra)]
    with poolcontext(processes=pool_size) as pool:
            results = pool.starmap(pre_processor_intrigoss, pool_arg_list)
        
    return results
'''

# In[33]:


f_i = []
f_p = []
f_a = []
fluxes_i = []
files_p = []
files_a = []
i_count = 0
i_count_tot = 0





# In[34]:


for root, dirs, files in os.walk(intrigoss_grid_path):
    for name in files:
        
       
        
        if i_count <= 6100 :
        
            i_count = i_count + 1
        
            teff_i,logg_i,feh_i,afe_i = getIntriParams(intrigoss_grid_path + name)
        
            if teff_i<= 5000 or logg_i <=3 or not(-1 < feh_i < 0):
                continue
            else:
                wav_intrigoss,flux_intrigoss = intrigoss_one_file(name)

            

            '''
            if i_count > i_batch_size:
                t_i = []
                t_i=pre_processor_intrigoss_parallel(wav_intrigoss,fluxes_i)
                for i in t_i:
                    f_i.append(i)
                    i_count_tot+=i_count
                    i_count = 0
                    fluxes_i = []
            '''

        #print(teff_i,logg_i,feh_i,afe_i)
        
            #print("Wavelength :"+str(len(wav_intrigoss)))
        
            print("Flux1 :"+str(len(flux_intrigoss)))
        
            matched_teff_p,matched_logg_p,matched_feh_p,matched_afe_p = find_closest_phoenix_match(teff_i, logg_i, feh_i, afe_i)
            matched_teff_a,matched_logg_a,matched_feh_a,matched_afe_a = find_closest_ambre_match(teff_i, logg_i, feh_i, afe_i)
        
            file_p = get_phoenix_filename(matched_teff_p, matched_logg_p, matched_feh_p, matched_afe_p)
            
        #print(file_p)
            file_a = get_ambre_filename(matched_teff_a, matched_logg_a, matched_feh_a, matched_afe_a)
            
            if file_p == '' or file_a == '':
                continue
            else:
                files_p.append(file_p)
                files_a.append(file_a)
            
            flux_intrigoss = pre_processor_intrigoss(wav_intrigoss,flux_intrigoss)
            print("Intrigoss:"+str(i_count))
            
            print("Flux2 :"+str(len(flux_intrigoss)))
            
            #print(i_count_tot)
            fluxes_i.append(flux_intrigoss)
        else:
            #i_count = i_count + 1
            continue

#print(files_a)
#print(files_p)

     


# In[22]:




def pre_processor_ambre_parallel(wav, fluxes):

    @contextmanager
    def poolcontext(*args, **kwargs):
            pool = multiprocessing.Pool(*args, **kwargs)
            yield pool
            pool.terminate()

    num_spectra = np.shape(fluxes)[0]
    num_cpu = multiprocessing.cpu_count()
    pool_size = num_cpu if num_spectra >= num_cpu else num_spectra
    print('[INFO] Pool size: {}'.format(pool_size))
    
    pool_arg_list = [(wav, fluxes[i])
                         for i in range(num_spectra)]
    with poolcontext(processes=pool_size) as pool:
            results = pool.starmap(pre_processor_ambre, pool_arg_list)
        
    return results


# In[23]:


def pre_processor_phoenix_parallel(wav, fluxes):

    @contextmanager
    def poolcontext(*args, **kwargs):
            pool = multiprocessing.Pool(*args, **kwargs)
            yield pool
            pool.terminate()

    num_spectra = np.shape(fluxes)[0]
    num_cpu = multiprocessing.cpu_count()
    pool_size = num_cpu if num_spectra >= num_cpu else num_spectra
    print('[INFO] Pool size: {}'.format(pool_size))
    
    pool_arg_list = [(wav, fluxes[i])
                         for i in range(num_spectra)]
    with poolcontext(processes=pool_size) as pool:
            results = pool.starmap(pre_processor_phoenix, pool_arg_list)
        
    return results


# In[24]:

Flux1 :57001
Flux1 :57001


/home/nrpu88/jupyter_py3/lib/python3.6/site-packages/astropy/stats/sigma_clipping.py:165: RuntimeWarning: invalid value encountered in greater
  _filtered_data.mask |= _filtered_data > max_value
/home/nrpu88/jupyter_py3/lib/python3.6/site-packages/astropy/stats/sigma_clipping.py:166: RuntimeWarning: invalid value encountered in less
  _filtered_data.mask |= _filtered_data < min_value


Intrigoss:10
Flux2 :39436
Flux1 :57001
Intrigoss:18
Flux2 :39436
Flux1 :57001
Intrigoss:23
Flux2 :39436
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Intrigoss:68
Flux2 :39436
Flux1 :57001
Flux1 :57001
Intrigoss:74
Flux2 :39436
Flux1 :57001
Flux1 :57001
Intrigoss:100
Flux2 :39436
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Intrigoss:283
Flux2 :39436
Flux1 :57001
Flux1 :57001
Flux1 :57001
Intrigoss:305
Flux2 :39436
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1 :57001
Intrigoss:405
Flux2 :39436
Flux1 :57001
Flux1 :57001
Intrigoss:420
Flux2 :39436
Flux1 :57001
Intrigoss:424
Flux2 :39436
Flux1 :57001
Flux1 :57001
Intrigoss:431
Flux2 :39436
Flux1 :57001
Flux1 :57001
Flux1 :57001
Flux1

In [12]:
p_count = 0
p_count_tot = 0

a_count = 0
a_count_tot = 0

fluxes_p = []
fluxes_a = []

blank_p = 0
blank_a = 0

for i in files_p:
    if i == '':
        blank_p = blank_p + 1

for j in files_a:
    if j == '':
        blank_a = blank_a + 1

count = 0
for i in fluxes_i:
    if i == '':
        count = count + 1

print("Intrigoss blank size:"+str(count))
print("Intrigoss size:"+str(len(fluxes_i)))


print("Phoenix size:" + str(len(files_p)))
print("Phoenix blank size:" + str(blank_p))


print("Ambre size:" + str(len(files_a)))
print("Ambre blank size:" + str(blank_a))



#files_p = [n.encode("ascii", "ignore") for n in files_p]
#files_a = [n.encode("ascii", "ignore") for n in files_a]

filename = 'intri_comp_fin_2'
save_path = '/home/nrpu88/StarNet-UVic/'+filename

with h5py.File(save_path, 'w') as f:  
    f.create_dataset('intrigoss_flux', data=np.asarray(fluxes_i))
    f.create_dataset('phoenix_files', (len(files_p),1),'S70', files_p)
    f.create_dataset('ambre_files', (len(files_a),1),'S70',files_a)    

f.close()



# In[ ]:

/home/nrpu88/jupyter_py3/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Intrigoss blank size:0
Intrigoss size:163
Phoenix size:163
Phoenix blank size:0
Ambre size:163
Ambre blank size:0
